In [1]:
import re
import pandas as pd
import os
import urllib

In [2]:
# Download data from ftp
urllib.request.urlretrieve("ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz", "/tmp/NASA_access_log_Jul95.gz")
urllib.request.urlretrieve("ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz","/tmp/NASA_access_log_Aug95.gz")

Out[112]: ('/tmp/NASA_access_log_Aug95.gz', <email.message.Message at 0x7f48c959b828>)

In [3]:
# Move data from local to DBFS
dbutils.fs.mv("file:/tmp/NASA_access_log_Jul95.gz", "dbfs:/NASA_access_log_Jul95.gz")
dbutils.fs.mv("file:/tmp/NASA_access_log_Aug95.gz", "dbfs:/NASA_access_log_Aug95.gz")

Out[113]: True

In [4]:
raw_data_files = ["NASA_access_log_Jul95.gz","NASA_access_log_Aug95.gz"]
base_df = spark.read.text(raw_data_files)
base_df.printSchema()
type(base_df)
base_df.show(10, truncate=False)

root
-- value: string (nullable = true)

+-----------------------------------------------------------------------------------------------------------------------+
value |
+-----------------------------------------------------------------------------------------------------------------------+
199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245 |
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985 |
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085 |
burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0 |
199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0 |
burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0 |
205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985 |
d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985 |
129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 200 7074 |
+-----------------------------------------------------------------------------------------------------------------------+
only showing top 10 rows

In [5]:
# Change Spark data to RDD data
base_df_rdd = base_df.rdd
type(base_df_rdd)
base_df_rdd.take(10)

Out[118]: [Row(value='199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'),
 Row(value='unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0'),
 Row(value='205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985'),
 Row(value='d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 200 7074')]

In [6]:
print((base_df.count(), len(base_df.columns)))

(3461613, 1)

In [7]:
# Check how data looks like
sample_logs = [item['value'] for item in base_df.take(15)]
sample_logs

Out[121]: ['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085',
 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0',
 '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0',
 '205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985',
 'd104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 200 7074',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /shuttle/countdown/count.gif HTTP/1.0" 200 40310',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:14 -0400] "GET /images/KSC-logosmall.gif HTTP/1.0" 200 1204',
 'd104.aa.net - - [01/Jul/1995:00:00:15 -0400] "GET /shuttle/countdown/count.gif HTTP/1.0" 200 40310',
 'd104.aa.net - - [01/Jul/1995:00:00:15 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786']

In [8]:
# Extract host
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
hosts = [re.search(host_pattern, item).group(1)
        if re.search(host_pattern, item)
        else 'not match'
        for item in sample_logs]
hosts

Out[122]: ['199.72.81.55',
 'unicomp6.unicomp.net',
 '199.120.110.21',
 'burger.letters.com',
 '199.120.110.21',
 'burger.letters.com',
 'burger.letters.com',
 '205.212.115.106',
 'd104.aa.net',
 '129.94.144.152',
 'unicomp6.unicomp.net',
 'unicomp6.unicomp.net',
 'unicomp6.unicomp.net',
 'd104.aa.net',
 'd104.aa.net']

In [9]:
# Extract Timestamp
timestamp_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})\]'
timestamps = [re.search(timestamp_pattern, item).group(1)
        if re.search(timestamp_pattern, item)
        else 'not match'
        for item in sample_logs]
timestamps

Out[123]: ['01/Jul/1995:00:00:01 -0400',
 '01/Jul/1995:00:00:06 -0400',
 '01/Jul/1995:00:00:09 -0400',
 '01/Jul/1995:00:00:11 -0400',
 '01/Jul/1995:00:00:11 -0400',
 '01/Jul/1995:00:00:12 -0400',
 '01/Jul/1995:00:00:12 -0400',
 '01/Jul/1995:00:00:12 -0400',
 '01/Jul/1995:00:00:13 -0400',
 '01/Jul/1995:00:00:13 -0400',
 '01/Jul/1995:00:00:14 -0400',
 '01/Jul/1995:00:00:14 -0400',
 '01/Jul/1995:00:00:14 -0400',
 '01/Jul/1995:00:00:15 -0400',
 '01/Jul/1995:00:00:15 -0400']

In [10]:
# Extract HTTP Request Method, URIs and Protocol
method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s(\S*)\"'
method_uri_protocol = [re.search(method_uri_protocol_pattern, item).groups()
                      if re.search(method_uri_protocol_pattern, item)
                      else "not match"
                      for item in sample_logs]
method_uri_protocol

Out[124]: [('GET', '/history/apollo/', 'HTTP/1.0'),
 ('GET', '/shuttle/countdown/', 'HTTP/1.0'),
 ('GET', '/shuttle/missions/sts-73/mission-sts-73.html', 'HTTP/1.0'),
 ('GET', '/shuttle/countdown/liftoff.html', 'HTTP/1.0'),
 ('GET', '/shuttle/missions/sts-73/sts-73-patch-small.gif', 'HTTP/1.0'),
 ('GET', '/images/NASA-logosmall.gif', 'HTTP/1.0'),
 ('GET', '/shuttle/countdown/video/livevideo.gif', 'HTTP/1.0'),
 ('GET', '/shuttle/countdown/countdown.html', 'HTTP/1.0'),
 ('GET', '/shuttle/countdown/', 'HTTP/1.0'),
 ('GET', '/', 'HTTP/1.0'),
 ('GET', '/shuttle/countdown/count.gif', 'HTTP/1.0'),
 ('GET', '/images/NASA-logosmall.gif', 'HTTP/1.0'),
 ('GET', '/images/KSC-logosmall.gif', 'HTTP/1.0'),
 ('GET', '/shuttle/countdown/count.gif', 'HTTP/1.0'),
 ('GET', '/images/NASA-logosmall.gif', 'HTTP/1.0')]

In [11]:
# Extract HTTP Status Code
status_pattern = r'\s(\d{3})\s'
status = [re.search(status_pattern, item).group(1) for item in sample_logs]
print(status)

['200', '200', '200', '304', '200', '304', '200', '200', '200', '200', '200', '200', '200', '200', '200']

In [12]:
# Extract HTTP Response Content Size
content_size_pattern = r'\s(\d+)$'
content_size = [re.search(content_size_pattern, item).group(1) for item in sample_logs]
print(content_size)

['6245', '3985', '4085', '0', '4179', '0', '0', '3985', '3985', '7074', '40310', '786', '1204', '40310', '786']

In [13]:
from pyspark.sql.functions import regexp_extract

logs_df = base_df.select(regexp_extract('value', host_pattern, 1).alias('host'),
                        regexp_extract('value', timestamp_pattern, 1).alias('timestamps'),
                        regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                        regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),
                        regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
                        regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
                        regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))
logs_df.show(10, truncate = True)
print((logs_df.count(), len(logs_df.columns)))

+--------------------+--------------------+------+--------------------+--------+------+------------+
 host| timestamps|method| endpoint|protocol|status|content_size|
+--------------------+--------------------+------+--------------------+--------+------+------------+
 199.72.81.55|01/Jul/1995:00:00...| GET| /history/apollo/|HTTP/1.0| 200| 6245|
unicomp6.unicomp.net|01/Jul/1995:00:00...| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|
 199.120.110.21|01/Jul/1995:00:00...| GET|/shuttle/missions...|HTTP/1.0| 200| 4085|
 burger.letters.com|01/Jul/1995:00:00...| GET|/shuttle/countdow...|HTTP/1.0| 304| 0|
 199.120.110.21|01/Jul/1995:00:00...| GET|/shuttle/missions...|HTTP/1.0| 200| 4179|
 burger.letters.com|01/Jul/1995:00:00...| GET|/images/NASA-logo...|HTTP/1.0| 304| 0|
 burger.letters.com|01/Jul/1995:00:00...| GET|/shuttle/countdow...|HTTP/1.0| 200| 0|
 205.212.115.106|01/Jul/1995:00:00...| GET|/shuttle/countdow...|HTTP/1.0| 200| 3985|
 d104.aa.net|01/Jul/1995:00:00...| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|
 129.94.144.152|01/Jul/1995:00:00...| GET| /|HTTP/1.0| 200| 7074|
+--------------------+--------------------+------+--------------------+--------+------+------------+
only showing top 10 rows

(3461613, 7)

In [14]:
(base_df
    .filter(base_df['value']
                .isNull())
    .count())

Out[128]: 0

In [15]:
# Even though there is no empty row, let's still check how many rows contain some missing values in any column
bad_rows_df = logs_df.filter(logs_df['host'].isNull()|
               logs_df['timestamps'].isNull()|
               logs_df['method'].isNull()|
               logs_df['endpoint'].isNull() |
               logs_df['status'].isNull() |
               logs_df['content_size'].isNull()|
               logs_df['protocol'].isNull())
print("There are {} rows contain some missing values in any of column".format(bad_rows_df.count()))

Out[129]: 33905

In [16]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum

def count_null(col_name):
    return spark_sum(col(col_name).isNull().cast('integer')).alias(col_name)

# Build up a list of column expressions, one per column.
exprs = [count_null(col_name) for col_name in logs_df.columns]

# Run the aggregation. The *exprs converts the list of expressions into
# variable function arguments.
logs_df.agg(*exprs).show()

+----+----------+------+--------+--------+------+------------+
host|timestamps|method|endpoint|protocol|status|content_size|
+----+----------+------+--------+--------+------+------------+
 0| 0| 0| 0| 0| 1| 33905|
+----+----------+------+--------+--------+------+------------+

In [17]:
# Find rows in base_df with null value in "status" column
null_status_df = base_df.filter(~base_df['value'].rlike(r'\s(\d{3})\s'))
null_status_df.count()

Out[25]: 1

In [18]:
null_status_df.show(truncate = False)

+--------+
value |
+--------+
alyssa.p|
+--------+

In [19]:
# Process other columns for null status rows
bad_status_df = null_status_df.select(regexp_extract('value', host_pattern, 1).alias('host'),
                                      regexp_extract('value', timestamp_pattern, 1).alias('timestamp'),
                                      regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                                      regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),
                                      regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
                                      regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
                                      regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))
bad_status_df.show(truncate=False)

+----+---------+------+--------+--------+------+------------+
host|timestamp|method|endpoint|protocol|status|content_size|
+----+---------+------+--------+--------+------+------------+
 | | | | |null |null |
+----+---------+------+--------+--------+------+------------+

In [20]:
# Seems like the row with null status is not informative, we can simply filter it out
logs_df = logs_df[logs_df['status'].isNotNull()] 
logs_df.count()

Out[134]: 3461612

In [21]:
# Check null value in each column again
exprs = [count_null(col_name) for col_name in logs_df.columns]
logs_df.agg(*exprs).show()

+----+----------+------+--------+--------+------+------------+
host|timestamps|method|endpoint|protocol|status|content_size|
+----+----------+------+--------+--------+------+------------+
 0| 0| 0| 0| 0| 0| 33904|
+----+----------+------+--------+--------+------+------------+

In [22]:
# Find rows in base_df with null value in "content_size" column
null_content_size_df = base_df.filter(~base_df['value'].rlike(r'\s(\d+)$'))
print(null_content_size_df.count())
null_content_size_df.take(10)

33905
Out[136]: [Row(value='dd15-062.compuserve.com - - [01/Jul/1995:00:01:12 -0400] "GET /news/sci.space.shuttle/archive/sci-space-shuttle-22-apr-1995-40.txt HTTP/1.0" 404 -'),
 Row(value='dynip42.efn.org - - [01/Jul/1995:00:02:14 -0400] "GET /software HTTP/1.0" 302 -'),
 Row(value='ix-or10-06.ix.netcom.com - - [01/Jul/1995:00:02:40 -0400] "GET /software/winvn HTTP/1.0" 302 -'),
 Row(value='ix-or10-06.ix.netcom.com - - [01/Jul/1995:00:03:24 -0400] "GET /software HTTP/1.0" 302 -'),
 Row(value='link097.txdirect.net - - [01/Jul/1995:00:05:06 -0400] "GET /shuttle HTTP/1.0" 302 -'),
 Row(value='ix-war-mi1-20.ix.netcom.com - - [01/Jul/1995:00:05:13 -0400] "GET /shuttle/missions/sts-78/news HTTP/1.0" 302 -'),
 Row(value='ix-war-mi1-20.ix.netcom.com - - [01/Jul/1995:00:05:58 -0400] "GET /shuttle/missions/sts-72/news HTTP/1.0" 302 -'),
 Row(value='netport-27.iu.net - - [01/Jul/1995:00:10:19 -0400] "GET /pub/winvn/readme.txt HTTP/1.0" 404 -'),
 Row(value='netport-27.iu.net - - [01/Jul/1995:00:10:28 -0400] "GET /pub/winvn/readme.txt HTTP/1.0" 404 -'),
 Row(value='dynip38.efn.org - - [01/Jul/1995:00:10:50 -0400] "GET /software HTTP/1.0" 302 -')]

In [23]:
# Seems like all null content_size mean no content_size, we can fillna with 0, which have the same meaning
logs_df = logs_df.na.fill({'content_size': 0})

In [24]:
# Check null value again
exprs = [count_null(col_name) for col_name in logs_df.columns]
logs_df.agg(*exprs).show()

+----+----------+------+--------+--------+------+------------+
host|timestamps|method|endpoint|protocol|status|content_size|
+----+----------+------+--------+--------+------+------------+
 0| 0| 0| 0| 0| 0| 0|
+----+----------+------+--------+--------+------+------------+

In [25]:
from pyspark.sql.functions import udf

month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

# Write an user-defined python function
def parse_clf_time(text):
    """ Convert Common Log time format into a Python datetime object
    Args:
        text (str): date and time in Apache time format [dd/mmm/yyyy:hh:mm:ss (+/-)zzzz]
    Returns:
        a string suitable for passing to CAST('timestamp')
    """
    # NOTE: We're ignoring the time zones here, might need to be handled depending on the problem you are solving
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(text[7:11]),
      month_map[text[3:6]],
      int(text[0:2]),
      int(text[12:14]),
      int(text[15:17]),
      int(text[18:20])
    )

In [26]:
# Check how timestamps column looks like
sample_ts = [item['timestamps'] for item in logs_df.select('timestamps').take(5)]
sample_ts

Out[139]: ['01/Jul/1995:00:00:01 -0400',
 '01/Jul/1995:00:00:06 -0400',
 '01/Jul/1995:00:00:09 -0400',
 '01/Jul/1995:00:00:11 -0400',
 '01/Jul/1995:00:00:11 -0400']

In [27]:
# Check how result looks like after applying user-defined python function
[parse_clf_time(item) for item in sample_ts]

Out[140]: ['1995-07-01 00:00:01',
 '1995-07-01 00:00:06',
 '1995-07-01 00:00:09',
 '1995-07-01 00:00:11',
 '1995-07-01 00:00:11']

In [28]:
# Change user-defined python function into sql function  by udf() in pyspark.sql module

udf_parse_time = udf(parse_clf_time)

logs_df = logs_df.select('*', udf_parse_time(logs_df['timestamps']).cast('timestamp').alias('time')).drop('timestamps')
logs_df.show(10, truncate=True)

+--------------------+------+--------------------+--------+------+------------+-------------------+
 host|method| endpoint|protocol|status|content_size| time|
+--------------------+------+--------------------+--------+------+------------+-------------------+
 199.72.81.55| GET| /history/apollo/|HTTP/1.0| 200| 6245|1995-07-01 00:00:01|
unicomp6.unicomp.net| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|1995-07-01 00:00:06|
 199.120.110.21| GET|/shuttle/missions...|HTTP/1.0| 200| 4085|1995-07-01 00:00:09|
 burger.letters.com| GET|/shuttle/countdow...|HTTP/1.0| 304| 0|1995-07-01 00:00:11|
 199.120.110.21| GET|/shuttle/missions...|HTTP/1.0| 200| 4179|1995-07-01 00:00:11|
 burger.letters.com| GET|/images/NASA-logo...|HTTP/1.0| 304| 0|1995-07-01 00:00:12|
 burger.letters.com| GET|/shuttle/countdow...|HTTP/1.0| 200| 0|1995-07-01 00:00:12|
 205.212.115.106| GET|/shuttle/countdow...|HTTP/1.0| 200| 3985|1995-07-01 00:00:12|
 d104.aa.net| GET| /shuttle/countdown/|HTTP/1.0| 200| 3985|1995-07-01 00:00:13|
 129.94.144.152| GET| /|HTTP/1.0| 200| 7074|1995-07-01 00:00:13|
+--------------------+------+--------------------+--------+------+------------+-------------------+
only showing top 10 rows

In [29]:
logs_df.printSchema()

root
-- host: string (nullable = true)
-- method: string (nullable = true)
-- endpoint: string (nullable = true)
-- protocol: string (nullable = true)
-- status: integer (nullable = true)
-- content_size: integer (nullable = false)
-- time: timestamp (nullable = true)

In [30]:
logs_df.limit(5).toPandas()

,host,method,endpoint,protocol,status,content_size,time
0,199.72.81.55,GET,/history/apollo/,HTTP/1.0,200,6245,1995-07-01 00:00:01
1,unicomp6.unicomp.net,GET,/shuttle/countdown/,HTTP/1.0,200,3985,1995-07-01 00:00:06
2,199.120.110.21,GET,/shuttle/missions/sts-73/mission-sts-73.html,HTTP/1.0,200,4085,1995-07-01 00:00:09
3,burger.letters.com,GET,/shuttle/countdown/liftoff.html,HTTP/1.0,304,0,1995-07-01 00:00:11
4,199.120.110.21,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,HTTP/1.0,200,4179,1995-07-01 00:00:11


In [31]:
# Ready to use dataframe for analysis, we need to cache it first
logs_df.cache()

Out[144]: DataFrame[host: string, method: string, endpoint: string, protocol: string, status: int, content_size: int, time: timestamp]

In [32]:
content_size_summary_df = logs_df.describe(['content_size'])
content_size_summary_df.toPandas()

,summary,content_size
0,count,3461612
1,mean,18928.844398216785
2,stddev,73031.47260949228
3,min,0
4,max,6823936


In [33]:
from pyspark.sql import functions as F
(logs_df.agg(F.min(logs_df['content_size']).alias('min_content_size'),
            F.max(logs_df['content_size']).alias('max_content_size'),
            F.mean(logs_df['content_size']).alias('mean_content_size'),
            F.stddev(logs_df['content_size']).alias('std_content_size'),
             F.count(logs_df['content_size']).alias('count_content_size')).toPandas())

,min_content_size,max_content_size,mean_content_size,std_content_size,count_content_size
0,0,6823936,18928.844398,73031.472609,3461612


In [34]:
status_freq_df = (logs_df
                 .groupBy('status')
                 .count()
                 .sort('status')
                 .cache())

In [35]:
print('Total distinct HTTP Status Codes:', status_freq_df.count())

Total distinct HTTP Status Codes: 8

In [36]:
status_freq_pd_df = (status_freq_df
                    .toPandas()
                    .sort_values(by=['count'],ascending=False))
status_freq_pd_df

,status,count
0,200,3100524
2,304,266773
1,302,73070
5,404,20899
4,403,225
6,500,65
7,501,41
3,400,15


In [37]:
!pip install -U seaborn

In [38]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

sns.catplot(x='status',
           y='count',
           data = status_freq_pd_df,
           kind = 'bar',
           order = status_freq_pd_df['status'])

In [39]:
log_freq_df = status_freq_df.withColumn('log(count)', F.log(status_freq_df['count']))
log_freq_df.show()

+------+-------+------------------+
status| count| log(count)|
+------+-------+------------------+
 200|3100524|14.947081687429097|
 302| 73070|11.199173164785263|
 304| 266773|12.494153388502301|
 400| 15| 2.70805020110221|
 403| 225| 5.41610040220442|
 404| 20899| 9.947456589918252|
 500| 65| 4.174387269895637|
 501| 41| 3.713572066704308|
+------+-------+------------------+

In [40]:
log_freq_pd_df = (log_freq_df
                    .toPandas()
                    .sort_values(by=['log(count)'],
                                 ascending=False))
sns.catplot(x='status', y='log(count)', data=log_freq_pd_df, 
            kind='bar', order=status_freq_pd_df['status'])

In [41]:
host_sum_df =(logs_df
               .groupBy('host')
               .count()
               .sort('count', ascending=False).limit(10))

host_sum_df.show(truncate=False)

+--------------------+-----+
host |count|
+--------------------+-----+
piweba3y.prodigy.com|21988|
piweba4y.prodigy.com|16437|
piweba1y.prodigy.com|12825|
edams.ksc.nasa.gov |11964|
163.206.89.4 |9697 |
news.ti.com |8161 |
www-d1.proxy.aol.com|8047 |
alyssa.prodigy.com |8037 |
 |7660 |
siltb10.orl.mmc.com |7573 |
+--------------------+-----+

In [42]:
host_sum_pd_df = host_sum_df.toPandas()
host_sum_pd_df.iloc[8]['host']

Out[155]: ''

In [43]:
paths_df = (logs_df
            .groupBy('endpoint')
            .count()
            .sort('count', ascending=False).limit(20))
paths_pd_df = paths_df.toPandas()
paths_pd_df

,endpoint,count
0,/images/NASA-logosmall.gif,208353
1,/images/KSC-logosmall.gif,164807
2,/images/MOSAIC-logosmall.gif,127648
3,/images/USA-logosmall.gif,126812
4,/images/WORLD-logosmall.gif,125668
5,/images/ksclogo-medium.gif,121278
6,/ksc.html,83679
7,/images/launch-logo.gif,75957
8,/history/apollo/images/apollo-logo1.gif,68856
9,/shuttle/countdown/,64689


In [44]:
not200_df = (logs_df.filter(logs_df['status']!=200))
error_endpoints_freq_df = (not200_df
                          .groupBy('endpoint')
                          .count()
                          .sort('count', ascending = False)
                          .limit(10))

In [45]:
error_endpoints_freq_df.show(truncate=False)

+---------------------------------------+-----+
endpoint |count|
+---------------------------------------+-----+
/images/NASA-logosmall.gif |40082|
/images/KSC-logosmall.gif |23763|
/images/MOSAIC-logosmall.gif |15245|
/images/USA-logosmall.gif |15142|
/images/WORLD-logosmall.gif |14773|
/images/ksclogo-medium.gif |13559|
/images/launch-logo.gif |8806 |
/history/apollo/images/apollo-logo1.gif|7489 |
/ |6296 |
/images/ksclogosmall.gif |5669 |
+---------------------------------------+-----+

In [46]:
unique_host_count = (logs_df
                     .select('host')
                     .distinct()
                     .count())
unique_host_count

Out[159]: 137933

In [47]:
host_day_distinct_df = logs_df.select(logs_df.host,
                            F.dayofmonth('time').alias('day')).drop_duplicates()
host_day_distinct_df.show(5, truncate = False)

+--------------------+---+
host |day|
+--------------------+---+
in24.inetnebr.com |1 |
d104.aa.net |1 |
199.72.81.55 |1 |
slppp6.intermind.net|1 |
199.120.110.21 |1 |
+--------------------+---+
only showing top 5 rows

In [48]:
daily_host_df = (host_day_distinct_df
                 .groupBy(['day'])
                 .count()
                .sort("day"))
daily_host_pd_df = daily_host_df.toPandas()

In [49]:
c = sns.catplot(x='day', y='count', 
                data=daily_host_pd_df, 
                kind='point', height=5, 
                aspect=1.5)

In [50]:
daily_hosts_df = (host_day_distinct_df
                     .groupBy('day')
                     .count()
                     .select(col("day"), col("count").alias("total_hosts")))
daily_hosts_df.show(10, truncate = False)

+---+-----------+
day|total_hosts|
+---+-----------+
31 |5913 |
28 |6090 |
26 |6741 |
27 |6846 |
12 |8135 |
22 |6828 |
1 |7609 |
13 |9567 |
6 |10133 |
16 |7143 |
+---+-----------+
only showing top 10 rows

In [51]:
total_daily_requests_df = (logs_df.select(F.dayofmonth("time").alias("day"))
                          .groupBy("day")
                          .count()
                          .select(col("day"), col("count").alias("total_reqs")))
      
avg_daily_reqests_per_host_df = total_daily_requests_df.join(daily_hosts_df, 'day')

avg_daily_reqests_per_host_df = (avg_daily_reqests_per_host_df.withColumn('avg_reqs', col("total_reqs")/col("total_hosts")).sort("day"))

avg_daily_reqests_per_host_pd_df = avg_daily_reqests_per_host_df.toPandas()

avg_daily_reqests_per_host_pd_df

,day,total_reqs,total_hosts,avg_reqs
0,1,98710,7609,12.972795
1,2,60265,4858,12.405311
2,3,130972,10238,12.792733
3,4,130009,9411,13.814579
4,5,126468,9640,13.119087
5,6,133380,10133,13.162933
6,7,144595,10048,14.390426
7,8,99024,7112,13.923510
8,9,95730,6699,14.290193
9,10,134108,8532,15.718237


In [52]:
c = sns.catplot(x = "day", y = "avg_reqs", data = avg_daily_reqests_per_host_pd_df,
               kind = "point", height = 5, aspect = 1.5)

In [53]:
# How many 404 records are in the log?
df_404 = logs_df.filter(logs_df['status']==404).cache()
print("There are {} 404 responses.".format(df_404.count()))

There are 20899 404 responses.

In [54]:
endpoints_404_count_df = (df_404.groupBy('endpoint')
                          .count()
                          .sort('count',ascending = False)
                          .limit(20))
endpoints_404_count_df.show(10, truncate = False)

+-----------------------------------------------------------------+-----+
endpoint |count|
+-----------------------------------------------------------------+-----+
/pub/winvn/readme.txt |2004 |
/pub/winvn/release.txt |1732 |
/shuttle/missions/STS-69/mission-STS-69.html |682 |
/shuttle/missions/sts-68/ksc-upclose.gif |426 |
/history/apollo/a-001/a-001-patch-small.gif |384 |
/history/apollo/sa-1/sa-1-patch-small.gif |383 |
/://spacelink.msfc.nasa.gov |381 |
/images/crawlerway-logo.gif |374 |
/elv/DELTA/uncons.htm |372 |
/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif|359 |
+-----------------------------------------------------------------+-----+
only showing top 10 rows

In [55]:
host_404_count_df = (df_404.groupBy('host')
                          .count()
                          .sort('count',ascending = False)
                          .limit(20))
host_404_count_df.show(10, truncate = False)

+---------------------------+-----+
host |count|
+---------------------------+-----+
hoohoo.ncsa.uiuc.edu |251 |
piweba3y.prodigy.com |157 |
jbiagioni.npt.nuwc.navy.mil|132 |
piweba1y.prodigy.com |114 |
 |112 |
www-d4.proxy.aol.com |91 |
piweba4y.prodigy.com |86 |
scooter.pa-x.dec.com |69 |
phaelon.ksc.nasa.gov |64 |
www-d1.proxy.aol.com |64 |
+---------------------------+-----+
only showing top 10 rows

In [56]:
daily_404_count_df = (df_404.groupBy(F.dayofmonth('time').alias('day'))
                          .count()
                          .sort('day',ascending = True))
daily_404_count_pd_df = daily_404_count_df.toPandas()
daily_404_count_pd_df

,day,count
0,1,559
1,2,291
2,3,778
3,4,705
4,5,733
5,6,1013
6,7,1107
7,8,691
8,9,627
9,10,713


In [57]:
c = sns.catplot(x='day',y='count',data=daily_404_count_pd_df,kind='point',height=5,aspect=1.5)

In [58]:
daily_404_count_df.sort("count",ascending=False).show(3)

+---+-----+
day|count|
+---+-----+
 7| 1107|
 6| 1013|
 25| 876|
+---+-----+
only showing top 3 rows

In [59]:
hourly_404_count_df = (df_404.groupBy(F.hour('time').alias('hour'))
                          .count()
                          .sort('hour',ascending = True))
hourly_404_count_pd_df = hourly_404_count_df.toPandas()
hourly_404_count_pd_df

,hour,count
0,0,774
1,1,648
2,2,868
3,3,603
4,4,351
5,5,307
6,6,269
7,7,458
8,8,705
9,9,840


In [60]:
c = sns.catplot(x="hour", y="count", data=hourly_404_count_pd_df,
               kind="bar",height=5,aspect=1.5)